Usualmente, todas las consultas SQL (a excepción de la sentencia insert) pueden incluir una cláusula condicional, la cual nos permite filtrarlas por ciertas condiciones que nosotros definamos. Para esto nosotros tenemos que utilizar la sentencia **where** para añadir uno o más condiciones para filtrar.
```sql
SELECT columna1 FROM tabla WHERE condicion
```

donde `condicion` puede ser cualquiera de los operadores condicionales:

- \> (Mayor), >= (Mayor o igual)
- < (Menor), <= (Menor o igual)
- = (Igual), != (No igual), <> (No igual)
- NOT (Negación)
- IN (especificar múltiples valores)
- BETWEEN (Especificar un rango de valores).

Por otra parte, podemos añadir múltiples condiciones utilizando los operadores `OR` y el operador `AND`. Entonces, reescribiendo la consulta anterior, a modo de ejemplo tenemos: 

```sql
SELECT * FROM tabla WHERE columna1 > 50;
SELECT columna1 FROM tabla WHERE columna2 > 50;
SELECT columna1, columna2 FROM tabla WHERE (columna2 < 10 AND columna1 > 5) OR columnaN = 1;
```

In [2]:
import sqlite3 #Importamos la libreria

#Generamos la conexion
conn = sqlite3.connect("db//Chinook_Sqlite.sqlite") #En este caso, corresponde al archivo Chinook_Sqlite
cursor = conn.cursor() #Generamos nuestro cursor para las consultas

Utilizando el siguiente comando, nosotros podremos ver todas las tablas existentes en nuestro archivo SQLite.

**Nota:** En la mayoría de los motores de bases de datos, existe una sentencia similar que permite enlistar todas las tablas a las cuales se pueden acceder

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';") #Seleccionamos la columna Nombre de la tabla sqlite_master, donde la columna type sea igual a 'table'
print(cursor.fetchall()) #Le pedimos a nuestro cursor que obtenga todos los resultados y los imprima:

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


Podemos ver entonces que existen 11 tablas en nuestra base de datos, cada una relacionada de distinta manera. Supongamos que queremos ver cuáles son las columnas que existen en una de estas tablas, donde nosotros utilizaremos la sentencia `PRAGMA`. Si bien, esta sentencia corresponde a operaciones más avanzadas, nosotros la podremos definir como: Una extensión de SQL para SQLite, la cual nos permite modificar la operación de SQLite, o en su defecto, consultar por datos internos de una base de datos.

Específicamente, nosotros utilizaremos el método implementado en `PRAGMA` llamado `table_info`, el cual permitirá imprimir las distintas columnas pertenecientes en una tabla. Esta consulta nos retornara distintos campos en el siguiente orden:

- Id de la columna
- Nombre de la columna
- Tipo de dato asociada a la columna
- Flag indicando si la columna puede ser nula o no
- Valor por defecto para dicha columna
- Flag de clave primaria


Mayor información la pueden revisar en la [documentación de PRAGMA](https://www.sqlite.org/pragma.html)

In [4]:
cursor.execute("PRAGMA table_info('Invoice')")
for element in cursor.fetchall():
    print(element) 

(0, 'InvoiceId', 'INTEGER', 1, None, 1)
(1, 'CustomerId', 'INTEGER', 1, None, 0)
(2, 'InvoiceDate', 'DATETIME', 1, None, 0)
(3, 'BillingAddress', 'NVARCHAR(70)', 0, None, 0)
(4, 'BillingCity', 'NVARCHAR(40)', 0, None, 0)
(5, 'BillingState', 'NVARCHAR(40)', 0, None, 0)
(6, 'BillingCountry', 'NVARCHAR(40)', 0, None, 0)
(7, 'BillingPostalCode', 'NVARCHAR(10)', 0, None, 0)
(8, 'Total', 'NUMERIC(10,2)', 1, None, 0)


Estos comandos son útiles cuando no tenemos un diagrama para una base de datos el cual podemos consultar de forma gráfica, el cómo están compuestas. En este caso, vemos que la tabla `Invoice`, está compuesta por 9 columnas, las cuales corresponden a distintos identificadores, y otras columnas relacionadas a un *invoice* (factura). A continuación, veremos un **modelo relacional** de cómo está compuesta la base de datos chinook y como se relacionan las distintas tablas.

<center><img src="https://drive.google.com/uc?id=10Ywd-W40s54shifso6U9BYPTYyMF5sYe"></center>

También existe la aplicación DB browser, que permite "ver" las bbdd

### Con todos estos antecedentes, procedamos a hacer consultas a la base de datos. 

Primero, veamos la tabla `Invoice`, seleccionemos los primeros 5 elementos utilizando el método `cursor.fetchmany(N)` donde `N = 5`

In [5]:
cursor.execute("SELECT * FROM Invoice")
registers = cursor.fetchmany(5)
for reg in registers:
    print(reg)

(1, 2, '2009-01-01 00:00:00', 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', 1.98)
(2, 4, '2009-01-02 00:00:00', 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', 3.96)
(3, 8, '2009-01-03 00:00:00', 'Grétrystraat 63', 'Brussels', None, 'Belgium', '1000', 5.94)
(4, 14, '2009-01-06 00:00:00', '8210 111 ST NW', 'Edmonton', 'AB', 'Canada', 'T6G 2C7', 8.91)
(5, 23, '2009-01-11 00:00:00', '69 Salem Street', 'Boston', 'MA', 'USA', '2113', 13.86)


In [6]:
cursor.execute("SELECT * FROM Invoice") #Ejecutamos la sentencia
all_registers = cursor.fetchall() #obtenemos todos los registros que cumplan la sentencia
print(len(all_registers)) #Imprimimos la cantiodad de elementos que existen

412


 ### ¿Cuantos registros existen en la tabla Invoice con un valor de factura mayor a 20?

In [7]:
cursor.execute("SELECT * FROM Invoice WHERE Total > 20")
response = cursor.fetchall()
print(response)
print("La cantidad de registros con un valor de factura mayor a 20 son:", len(response))

[(96, 45, '2010-02-18 00:00:00', 'Erzsébet krt. 58.', 'Budapest', None, 'Hungary', 'H-1073', 21.86), (194, 46, '2011-04-28 00:00:00', '3 Chatham Street', 'Dublin', 'Dublin', 'Ireland', None, 21.86), (299, 26, '2012-08-05 00:00:00', '2211 W Berry Street', 'Fort Worth', 'TX', 'USA', '76110', 23.86), (404, 6, '2013-11-13 00:00:00', 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', 25.86)]
La cantidad de registros con un valor de factura mayor a 20 son: 4


### ¿Cuantos registros existen en la tabla Invoice con un valor de factura menor a 10?

In [8]:
cursor.execute("SELECT * FROM Invoice WHERE Total < 10")
print("La cantidad de registros con un valor de factura menor a 10 son:", len(cursor.fetchall()))

La cantidad de registros con un valor de factura menor a 10 son: 348


### ¿Cuantas canciones pertenecen al Género Musical con identificador igual a 1?

In [9]:
cursor.execute("SELECT * FROM Track WHERE GenreId = 1")
print("La cantidad de canciones con el identificador de Genero Musical igual a 1 son:",len(cursor.fetchall()))

La cantidad de canciones con el identificador de Genero Musical igual a 1 son: 1297


### ¿Cuantas canciones pertenecen al Género Musical con identificador distinto de 1?

In [10]:
cursor.execute("SELECT * FROM Track WHERE GenreId != 1")
print("La cantidad de canciones con el identificador de Genero Musical distinto a 1 son:",len(cursor.fetchall()))

La cantidad de canciones con el identificador de Genero Musical distinto a 1 son: 2206


Alternativamente, podemos utilizar el operador NOT a la condición lógica de igualdad, para poder hacer el mismo proceso de consultar la última consulta de ¿Cuantas canciones pertenecen al Género Musical con identificador distinto de 1? La sintaxis corresponde a la siguiente:

In [13]:
cursor.execute("SELECT * FROM Track WHERE NOT (GenreId = 1)")
print("Alternativamente, utilizando el operador NOT:",len(cursor.fetchall())) # Vamos a ver que las cantidades coinciden 

Alternativamente, utilizando el operador NOT: 2206


Para los operadores `IN` y `BETWEEN` la sintaxis puede ser distinta a una consulta típica de SQL, donde estas son:
```SQL
SELECT * FROM Tabla WHERE Columna IN (Val1,Val2,...,ValN) -- Uso del IN
SELECT * FROM Tabla WHERE Columna BETWEEN Val1 AND Val2 -- Uso del BETWEEN
```

¿Cuantas canciones existen las cuales pertenezcan a los géneros musicales definidos por los identificadores 1, 2 o 3?

In [14]:
cursor.execute("SELECT * FROM Track WHERE GenreId IN (1,2,3)")
print(len(cursor.fetchall()))

1801


¿Cuantas canciones existen las cuales tengan una duración entre 100000 y 300000 Milisegundos?

In [15]:
cursor.execute("SELECT * FROM Track WHERE Milliseconds BETWEEN 100000 AND 300000")
print(len(cursor.fetchall()))

2376


In [16]:
cursor.execute("SELECT * FROM Track WHERE Milliseconds NOT BETWEEN 100000 AND 300000")
print(len(cursor.fetchall()))

1127
